**Table of contents**<a id='toc0_'></a>    
- [Data Preprocessing](#toc1_)    
- [Modelling](#toc2_)    
- [Evaluate & Export Model](#toc3_)    
- [Metrics](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Data Preprocessing](#toc0_)

In [1]:
# Constants & Hyperparameters to define
RANDOM_SEED = 42

NUM_WORDS = 5000
MAX_SEQ_LEN = 50
EMBEDDING_DIM = 50
NUM_FILTERS = 64
KERNEL_SIZE = 5
NUM_CLASSES = 2
SAMPLE_FRAC = 0.5


# Import Libraries
import pandas as pd
from tqdm.notebook import tqdm

# Import Functions
import sys
sys.path.append('../src')
from preprocessing import DataInspection, HandleMissingValues, RemoveDuplicates, TranslateText, HandleOutliers, SplitDataset



data_inspection = DataInspection()
handle_missing_values = HandleMissingValues()
remove_duplicates = RemoveDuplicates()
translate_text = TranslateText()
handle_outliers = HandleOutliers()
split_dataset = SplitDataset()



data_inspection.set_next(handle_missing_values)
handle_missing_values.set_next(remove_duplicates)
remove_duplicates.set_next(translate_text)
translate_text.set_next(handle_outliers)
handle_outliers.set_next(split_dataset)



raw_data = pd.read_csv('../data/raw_data.csv')
print(f'raw_data.shape: {raw_data.shape}')

######################################################################
# Sample the data
raw_data = raw_data.sample(frac=SAMPLE_FRAC, 
    random_state=RANDOM_SEED, 
    ignore_index=True)

print(f'raw_data.shape: {raw_data.shape}')
######################################################################

# Ejecutar la pipeline
X_train, X_test, y_train, y_test = data_inspection.process(raw_data)

2024-07-09 17:17:06.139619: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-09 17:17:06.148518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 17:17:06.160808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 17:17:06.160834: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 17:17:06.168889: I tensorflow/core/platform/cpu_feature_gua

raw_data.shape: (4000000, 3)
raw_data.shape: (2000000, 3)

**View Data Structure**

HEAD


,labels,review_title,text
0,__label__1,"Deeply disappointing, faulty morality & social...","The book delves very little into art, aside fr..."
1,__label__2,insight into the philosophy of libertarian soc...,"In ""The Limits of State Action"" Enlightenment ..."
2,__label__2,a great book,"""In vain did the Bedouins strive to cut down a..."
3,__label__1,toys for great sex,"wow, that was bad, I threw it away after watch..."
4,__label__2,i love this movie!!!!,i just finished reading Someone Like You and i...



SAMPLE


,labels,review_title,text
1256441,__label__2,one of the Top 100 Albums Of All Time!,"just when you thought ""gentleman take polaroid..."
624201,__label__2,Great,"This works great, so much better than any othe..."
1665824,__label__2,"Skylark, (Sarah Plain and Tall)",I purchased this for my sons 3rd grade class. ...
309255,__label__2,L'Anza Will Get You Through Wrecked Hair Days,My stylist left straightening chems on my high...
212623,__label__1,hmmmm,can i get back the time i wasted watching this...


...


**Info Summary**

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 3 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   labels        object
 1   review_title  object
 2   text          object
dtypes: object(3)
memory usage: 1.1 GB

**Summary Statistics**


,labels,review_title,text
count,2000000,1995250,2000000
unique,2,1501467,1998354
top,__label__1,Disappointed,"I bought a QPS back in April, I had configurat..."
freq,1000639,5274,10



**Handle Missing Values**

review_title    4750
labels             0
text               0
dtype: int64

Actual Missing Values

labels          0
review_title    0
text            0
dtype: int64

**Remove Duplicates**

There are no duplicated values.
ACTUAL SHAPE: (2000000, 3)

**Translate Text**


Translating Text: 100%|██████████| 2000000/2000000 [1:22:14<00:00, 405.34it/s]  



**Handle Outliers**

Number of detected outliers: 296
SHAPE BEFORE: (2000000, 4)
ACTUAL SHAPE: (1999704, 4)


LABELS:
1.0

X_train shape: (1599763, 50)
X_test shape: (399941, 50)
y_train shape: (1599763,)
y_test shape: (399941,)


# <a id='toc2_'></a>[Modelling](#toc0_)

**Sequential convolutional neural network (CNN) for text classification**

1. Embedding Layer
* `Embedding(input_dim=5000, output_dim=100, input_length=100)`
* `input_dim=5000`: This specifies the vocabulary size, meaning the model can handle up to **5000** unique words.
* `output_dim=100`: This defines the dimensionality of the embedding vector, which compresses each word into a **100**-dimensional vector.
* `input_length=100`: This sets the maximum length of the input text sequences (sentences or paragraphs) to **100** words.

2. Convolutional Layer
* `Conv1D(filters=64, kernel_size=5, activation='relu')`: This 1D convolutional layer extracts features from the embedded text sequences.
* `filters=64`: This indicates the number of filters used to identify patterns in the text.
* `kernel_size=5`: This defines the size of the window that the filter slides over the text sequence (**5** words in this case).
* `activation='relu'`: This activation function introduces non-linearity, allowing the model to learn complex relationships between words.

    * `'relu'` means Rectified Linear Unit (ReLU). 
    * For any input value $(x)$, it outputs the value itself if it's positive $(x > 0)$ and zero otherwise $(x <= 0)$. 
    * Mathematically, it can be represented as:
    * $f(x) = max(0, x)$

3. Pooling Layer
* `MaxPooling1D(pool_size=4)`: This layer reduces the dimensionality of the data by taking the maximum value from every window of size **4** along the sequence This helps control overfitting and focuses on the most important features.

4. Flattening Layer
* `Flatten()`: This layer transforms the 2D output from the convolutional layer into a 1D vector suitable for feeding into the fully connected layers.

5. Fully Connected Layers
* `Dense(10, activation='relu')`: This first fully connected layer has **10** neurons and uses the ReLU activation function. It learns higher-level features by combining the extracted features from the convolutional layers.

* `Dense(3, activation='softmax')`: This final fully connected layer has 3 neurons and uses the softmax activation function. It outputs a probability distribution over 3 categories, making it suitable for multi-class classification tasks (e.g., classifying text into 3 different genres).

    * `'softmax'`: For each element $(i)$ in the input vector, softmax calculates the probability $(p_i)$ using the following formula:
    * $p_i = exp(x_i) / Σ(exp(x_j))$  for all $j$ in the vector
    * Here, $exp(x_i)$ represents the exponentiation of the i-th element in the input vector.
    * $Σ(exp(x_j))$ represents the sum of the exponentials of all elements in the vector.

6. Compiling the Model:
* `model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])`: This compiles the model by specifying the optimizer (Adaptive Moment Estimation (Adam) for efficient training), the loss function (sparse categorical crossentropy for multi-class classification), and the metrics (accuracy to measure performance).

In [2]:
# Import Libraries
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

# Disable XLA JIT Compilation
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'



padded_sequences = tf.convert_to_tensor(X_train, dtype=tf.float32)
labels = tf.convert_to_tensor(y_train, dtype=tf.float32)

print('\n')
print(type(padded_sequences))
print(padded_sequences.dtype)
print(padded_sequences[0])
print('\n')
print(type(labels))
print(labels.dtype)
print(labels[0])
print('\n')



# Define the CNN model
model = Sequential([
    Embedding(
        input_dim=MAX_SEQ_LEN, 
        output_dim=EMBEDDING_DIM, 
        input_length=MAX_SEQ_LEN),
    Conv1D(
        filters=NUM_FILTERS, 
        kernel_size=KERNEL_SIZE, 
        activation='relu', 
        padding='same'),
    MaxPooling1D(
        pool_size=4, 
        padding='same'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', 
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall'])



# Fit the model    
model.fit(x=padded_sequences, 
    y=labels, 
    epochs=10, 
    batch_size=32, 
    validation_split=0.2)

# with tf.device('/CPU:0'):
#     model.fit(x=padded_sequences, 
#         y=labels, 
#         epochs=10, 
#         batch_size=32, 
#         validation_split=0.2)

2024-07-09 18:43:04.807797: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-09 18:43:04.807822: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: heroines
2024-07-09 18:43:04.807828: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: heroines
2024-07-09 18:43:04.807916: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 550.90.7
2024-07-09 18:43:04.807936: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 550.90.7
2024-07-09 18:43:04.807941: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 550.90.7




<class 'tensorflow.python.framework.ops.EagerTensor'>
<dtype: 'float32'>
tf.Tensor(
[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 1.216e+03 1.000e+01 6.240e+02 4.900e+01
 1.500e+01 1.270e+02 8.300e+01 2.290e+02 1.660e+02 2.660e+02 3.000e+00
 5.800e+01 7.770e+02 1.000e+01 2.900e+01 4.000e+00 1.216e+03 2.200e+01
 1.242e+03 1.120e+02 1.500e+03 3.120e+02 7.000e+00 1.000e+01 1.000e+00
 4.160e+02 1.449e+03 1.000e+01 4.680e+02 4.500e+01 6.000e+00 1.132e+03
 1.216e+03 3.900e+01 1.000e+00 1.690e+02 1.516e+03 6.000e+00 2.754e+03
 1.449e+03], shape=(50,), dtype=float32)


<class 'tensorflow.python.framework.ops.EagerTensor'>
<dtype: 'float32'>
tf.Tensor(0.0, shape=(), dtype=float32)


Epoch 1/10
39995/39995 ━━━━━━━━━━━━━━━━━━━━ 60s 1ms/step - accuracy: 0.6938 - loss: 0.5684 - precision: 0.6827 - recall: 0.7278 - val_accuracy: 0.7284 - val_loss: 0.5267 - val_precision: 0.7234 - val_recall: 0.7401
Epoch 2/10
39995/39995 ━━━━━━━━━━━━━━━━━━━━ 

# <a id='toc3_'></a>[Evaluate & Export Model](#toc0_)

$F1-score=2×(Precision+Recall)/(Precision×Recall)​$

In [3]:
model_loss, model_accuracy, model_precision, model_recall = model.evaluate(X_test, y_test)

# Calculate the F1-score
if (model_precision + model_recall) == 0:
    model_f1_score = 0
else:
    model_f1_score = 2 * (model_precision * model_recall) / (model_precision + model_recall)

print(f'Accuracy:\t {model_accuracy:.4f}')
print(f'Precision:\t {model_precision:.4f}')
print(f'Recall:\t\t {model_recall:.4f}')
print(f'F1-score:\t {model_f1_score:.4f}')

# Export model
model.export('../model')

'''
# Later, in a different process / environment...
reloaded_artifact = tf.saved_model.load("path/to/location")
predictions = reloaded_artifact.serve(input_data)
''';

12499/12499 ━━━━━━━━━━━━━━━━━━━━ 9s 704us/step - accuracy: 0.7442 - loss: 0.5084 - precision: 0.7496 - recall: 0.7345
Accuracy:	 0.7438
Precision:	 0.7481
Recall:		 0.7349
F1-score:	 0.7415
INFO:tensorflow:Assets written to: ../model/assets


INFO:tensorflow:Assets written to: ../model/assets


Saved artifact at '../model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138482462123280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138482462250784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138482462507600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138482481486480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138482481810416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138482481912992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138482482278992: TensorSpec(shape=(), dtype=tf.resource, name=None)


# <a id='toc4_'></a>[Metrics](#toc0_)

**Sample 0.0001**:

$0.5750$ - Accuracy         
$0.5000$ - Precision        
$0.1176$ - Recall       
$0.1905$ - F1-score     

**Sample 0.001**:
      
$0.5938$ - Accuracy     
$0.5610$ - Precision        
$0.5806$ - Recall     
$0.5707$ - F1-score

**Sample 0.01**:

$0.6658$ - Accuracy         
$0.6552$ - Precision        
$0.6726$ - Recall       
$0.6638$ - F1-score     

**Sample 0.1**:

$0.7249$ - Accuracy         
$0.7284$ - Precision        
$0.7161$ - Recall       
$0.7222$ - F1-score    

**Sample 0.2**:

$0.7377$ - Accuracy         
$0.7338$ - Precision        
$0.7451$ - Recall       
$0.7394$ - F1-score  

**Sample 0.5**:

$0.7438$ - Accuracy         
$0.7481$ - Precision        
$0.7349$ - Recall       
$0.7415$ - F1-score  